# 文本处理测试

解压缩文件

In [0]:
'''
import zipfile

filePath = '/content/drive/My Drive/Colab Notebooks/深度学习/wed/04-第四课 卷积神经网络/04-第四课第四周编程作业.zip'
outPath = '/content/drive/My Drive/Colab Notebooks/深度学习/wed/04-第四课 卷积神经网络'
z = zipfile.ZipFile(filePath, 'r')
z.extractall(path=outPath)
z.close()
'''

安装结巴分词器

In [0]:
#!pip install jieba

In [0]:
#import jieba

#seg_list = jieba.cut("我家在河南省郑州市", cut_all=False)
#print("Default Mode: " + "/".join(seg_list))  # 精确模式

读取数据

In [0]:
'''
with open("/content/drive/My Drive/数据集/商品描述文案数据集/item_desc_dataset.txt", "r") as f:
  data = f.readline()
  seg_list = jieba.cut(data, cut_all=False)
  #print(data)
  print("/".join(seg_list))
'''

'''
f = open("/content/drive/My Drive/数据集/商品描述文案数据集/item_desc_dataset.txt", "r")
maxLen = 0
dataLen = 0
while True:
  line = f.readline()
  sAry = line.split('\t',1)
  seg_list = jieba.cut(sAry[1], cut_all=False)
  #print(sAry[1])
  #print("/".join(seg_list))
  #print(len(list(seg_list)))
  maxLen = max(maxLen,len(list(seg_list)))
  dataLen = dataLen + 1
  print(maxLen)
  if not line:
    break
print('记录条数为:',dataLen)
print('句子最长为:',maxLen)
f.close()
'''

读取预训练词向量

In [0]:
'''
ilePath = '/content/drive/My Drive/Colab Notebooks/RNN/预训练词向量/sgns.target.word-word.dynwin5.thr10.neg5.dim300.iter5'
f = open(filePath, "r")
while True:
  line = f.readline()
  print(line)
  if not line:
    break
'''    

# 导入类库

In [0]:
import collections
import time
import numpy as np
import random
import math

try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

# 古诗数据处理

读取古诗,将古诗存在数组中

In [0]:
poetry_file = '/content/drive/My Drive/数据集/古诗文本/poetry.txt'
# 诗集  
poetrys = []
filterSAry = ["_", "(", "《","["]
index = 0  
with open(poetry_file, "r") as f:  
    for line in f: 
      line = line.strip('\n')
      lineAry = line.strip(' ').split(':')
      title = lineAry[0]
      content = lineAry[1]

      if '_' in content or '（' in content or '《' in content or '[' in content:
        continue
      if len(content) < 5 or len(content) > 79:
        continue
      
      content = content + ']'
      poetrys.append(content)

按诗的字数排序

In [0]:
poetrys = sorted(poetrys,key=lambda poetry:len(poetry))
print('诗的总数为:',len(poetrys))

统计每首诗的长度,并统计相同长度的诗都各有多少首，统计出哪种长度的诗个数最多

In [0]:
poetrys_length = [len(poetry) for poetry in poetrys]
counter = collections.Counter(poetrys_length)
poetrys_length,poetrys_num = zip(*counter.items())
#求出最多个数的索引
poertyLength_maxNum_index = np.argmax(poetrys_num)
#print(counter)
print('数量最多的诗的长度为:',poetrys_length[poertyLength_maxNum_index])
print('数量最多的诗的个数为:',poetrys_num[poertyLength_maxNum_index])

将个数最多的诗装进列表

In [0]:
poetrys_train = [poetry for poetry in poetrys if len(poetry) == 49]
print('将要参与训练的诗的个数为:',len(poetrys_train))

统计每个字出现的次数

In [0]:
# 统计每个字出现次数
start = time.clock()
all_words = []
for poetry in poetrys_train:
  all_words += [word for word in poetry]
#words = list(set(all_words))
words_count = collections.Counter(all_words)
words_count = sorted(
    words_count.items(), 
    key=lambda x: x[1],
    reverse=True)
words,_ = zip(*words_count)
end = time.clock()
print('一共有',len(words),'个字符')
print('用时:',end - start)
print(words[:10])
#print(list(set(result)).sort(key=result.))    

# 构造训练数据集

定义初始变量，文本的长度为49,参与训练的样本为12530个

In [0]:
train_num = 12530
maxlen = 49

In [0]:
#将字符映射为数组，并将数字映射为字符
char_to_ix = {ch:i+1 for i,ch in enumerate(words)}
ix_to_char = {i+1:ch for i,ch in enumerate(words)}

'''
print(ix_to_char)
print(ix_to_char[4])
print(char_to_ix)
print(char_to_ix['不'])
'''

In [0]:
'''
import random
print(poetrys_train[0])
random.shuffle(poetrys_train)
print(poetrys_train[0])
'''

In [0]:
#构造一个生成器
#x shape为 (batch,maxlen - 1,len(words)+1)
#y shape为 (batch,maxlen - 1,len(words)+1)
def gen_generator():
  dataLength = len(poetrys_train)
  batch = 32
  while True:
    random.shuffle(poetrys_train)
    data_batch = np.zeros((32, maxlen))
    for i in range(dataLength):
      for wix,word in enumerate(poetrys_train[i]):
        data_batch[i%batch,wix] = char_to_ix[word]
      if (i%(batch-1)) == 0 and i != 0:
        data_baatch_oneHot = tf.keras.utils.to_categorical(data_batch,num_classes=len(words) + 1)
        x = data_baatch_oneHot[:,:maxlen - 1,:]
        y = data_baatch_oneHot[:,1:,:]
        yield (x,y)

'''
gen = gen_generator()
A = next(gen)
print(A[0].shape)
print(A[1].shape)
print(A[2].shape)
'''

# 构造模型

In [0]:
#构造模型
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(
    1024,
    return_sequences = True,
    input_shape=(maxlen - 1,len(words)+1)))
#model.add(tf.keras.layers.LSTM(128,return_sequences = True))
model.add(tf.keras.layers.Dense(len(words) + 1,activation='softmax'))
#model.add(tf.keras.layers.LSTM(len(words) + 1))

model.compile(loss='categorical_crossentropy',optimizer='adam')

In [0]:
model.summary()

# 训练模型

In [0]:
#训练模型
gen = gen_generator()
train_epochs = 100
stepsPerEpochs = math.floor(len(poetrys_train)/32)
model.fit_generator(gen,
          epochs=train_epochs,
          steps_per_epoch=stepsPerEpochs)

# 保存模型

In [0]:

#模型保存
modelPath = '/content/drive/My Drive/myModel/rnn-poetrys-100.h5'
model.save(modelPath)


# 加载训练好的模型进行测试

In [0]:

#加载模型
modelPath = '/content/drive/My Drive/myModel/rnn-poetrys-100.h5'
model = tf.keras.models.load_model(modelPath)
model.summary()


In [0]:
#计算出第一个字是什么
#x shape为 (batch,maxlen - 1,len(words)+1)
#y shape为 (batch,maxlen - 1,len(words)+1)
x = np.zeros(shape=(1,maxlen - 1,len(words) + 1))
y = model.predict(x)
firstWord_oneHot = y[0,0,:]
print(firstWord_oneHot.shape)
idx = np.argmax(firstWord_oneHot,axis=-1)
firstWord = ix_to_char[idx]
print(firstWord)

In [0]:
print(maxlen - 1)

In [0]:
#循环的出每一个字
genPoetry = []
genPoetry.append(firstWord)
xPoetry = np.zeros(shape=(1,maxlen - 1,len(words) + 1))
xPoetry[0,0] = firstWord_oneHot
#print(np.argmax(xPoetry[0,0]))
for i in range(maxlen - 2):
  y = model.predict(xPoetry)
  if i < (maxlen -1):
    xPoetry[0,i+1] = y[0,i,:]
  idxTmp = np.argmax(y[0,i,:],axis=-1)
  genPoetry.append(ix_to_char[idxTmp])

In [0]:
print("生成的诗为：")
print("".join(genPoetry))

In [0]:
T = np.random.randint(low=0,high=2,size=(5,6))
print(T)
print(T[:,:5])

In [0]:
print(poetrys_train[0])
print(char_to_ix['随'])
#print(ix_to_char[508])

In [0]:
A = np.random.randint(low=0,high=10,size=(8,6,5))
B = tf.keras.utils.to_categorical(A,num_classes=10)
print(A)
print(A[0])
print(B,B.shape)

In [0]:
# 取前多少个常用字
words = words[:len(words)] + (' ',)

In [0]:
# 每个字映射为一个数字ID
word_num_map = dict(zip(words,range(len(words))))
print(word_num_map)

In [0]:
# 把诗转换为向量形式，参考TensorFlow练习1
to_num = lambda word: word_num_map.get(word, len(words))
poetrys_vector = [ list(map(to_num, poetry)) for poetry in poetrys]